In [ ]:
import numpy as np
import networkx as nx
import scipy
from karateclub.graph_embedding import Graph2Vec
import torch
from qtensor import QAOA_energy

In [ ]:
def matrices_to_graphs(matrix_list):
    '''
    This function takes a list of adjacency matrices and generates a list of graphs.
    '''

    g_list = []
    for matrix in matrix_list:
        array = np.array(matrix)
        g = nx.from_numpy_array(array)
        g_list.append(g)

    return g_list

In [ ]:
# Import training graph list
training_file = open("29260_40_node_random_graphs.txt")
training_matrix_list = np.loadtxt(training_file).reshape(29260, 40, 40)
training_graph_list = matrices_to_graphs(training_matrix_list)

In [ ]:
# Import testing graph list
test_file = open("36_40_node_random_graphs.txt")
test_matrix_list = np.loadtxt(test_file).reshape(36, 40, 40)
test_graph_list = matrices_to_graphs(test_matrix_list)

In [ ]:
# Train Graph2Vec model (the number of epochs and learning rate hyperparameters are chosen for this particular training set)
model = Graph2Vec(epochs=100, learning_rate=0.065)
model.fit(training_graph_list)
model_array = model.get_embedding()

In [ ]:
def find_indices(vector):
    '''
    This function returns the indices from the first three closest and last three furthest graphs
    in embedded space to the test graph. The first index in the array corresponds to the graph that
    is most similar, while the last index corresponds to the most dissimilar.
    '''
    
    length = len(vector)
    sorted_vector = sorted(vector)
    
    max_value = sorted_vector[length - 1]
    second_max_value = sorted_vector[length - 2]
    third_max_value = sorted_vector[length - 3]
    
    min_value = sorted_vector[0]
    second_min_value = sorted_vector[1]
    third_min_value = sorted_vector[2]
    
    for i in range(len(vector)):
        if vector[i] == max_value:
            max_index = vector.index(vector[i])
        if vector[i] == second_max_value:
            second_max_index = vector.index(vector[i])
        if vector[i] == third_max_value:
            third_max_index = vector.index(vector[i])
        if vector[i] == min_value:
            min_index = vector.index(vector[i])
        if vector[i] == second_min_value:
            second_min_index = vector.index(vector[i])
        if vector[i] == third_min_value:
            third_min_index = vector.index(vector[i])
        else:
            continue
            
    indices = [min_index, second_min_index, third_min_index, third_max_index, second_max_index, max_index]
    
    return indices

In [ ]:
def euclidean_distance(model_vector, infer_vector):
    '''
    This function calculates the Euclidean distance between two (graph) embedded vectors.
    '''
    
    diffs = []
    
    for i in range(len(model_vector)):
        
        diff = (model_vector[i] - infer_vector[0][i])**2
        diffs.append(diff)
    
    return np.sqrt(sum(diffs))

In [ ]:
# Find indices 
indices = []

for i in range(len(test_graph_list)):
    infer_vector = model.infer([test_graph_list[i]])
    euclidean_distances = []
    for j in range(len(model_array)):
        dist = euclidean_distance(model_array[j], infer_vector)
        euclidean_distances.append(dist)
    index = find_indices(euclidean_distances)
    indices.append(index)

In [ ]:
# Import optimal parameters from graphs in the training set
gamma_params_file = open('training_optimal_gammas.txt')
gamma_params = np.loadtxt(gamma_params_file).reshape(29260, 20, 3)

beta_params_file = open('training_optimal_betas.txt')
beta_params = np.loadtxt(beta_params_file).reshape(29260, 20, 3)

In [ ]:
# Transfer parameters and compute average energy (over 20 sets of parameters)

test_file = open("36_40_node_random_graphs.txt")
test_matrix_list = np.loadtxt(test_file).reshape(36, 40, 40)
test_graph_list = matrices_to_graphs(test_matrix_list)

# Test graphs have to be imported again (for some reason when running the model, graphs get turned into different objects)

index = 6 # 3 top indices 3 bottom indices (you can modify this if you only want to run, say, only top indices)
multistarts = 20

average_transfer_energies = [[0 for x in range(index)] for y in range(multistarts)]

for i in range(len(test_graph_list)):
    for j in range(index):
        transfer_energies = []
        for k in range(multistarts):
            transfer_energy = QAOA_energy(test_graph_list[i], gamma=gamma_params[indices[i][j]][k], beta=beta_params[indices[i][j]][k])
            transfer_energies.append(transfer_energy)
            
        average_transfer_energy = np.average(transfer_energies)
        average_transfer_energies[i][j] = average_transfer_energy